<a href="https://colab.research.google.com/github/hssitja/PhD-Dissertation/blob/Chapter-3/Chapter4_Empirical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install
!pip install stargazer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import
from google.colab import auth
# from google.cloud import bigquery
# from google.colab import data_table

from google.colab import drive, auth
drive.mount('/content/drive')

import os
path = '/content/drive/My Drive/4 Vaccination and Election Results/Data'
os.chdir(path)

# project = 'phd-dissertation-343113' # Project ID inserted based on the query results selected to explore
# location = 'US' # Location inserted based on the query results selected to explore
# client = bigquery.Client(project=project, location=location)
# data_table.enable_dataframe_formatter()
# auth.authenticate_user()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from stargazer.stargazer import Stargazer

# from zipfile import ZipFile

SEED = np.random.seed(42)

with open('/content/drive/MyDrive/2 Essay Bayesian dinamic model for vaccine hesitancy/Python/Google developer/config') as f:
    api_key = f.readline()
    f.close

# auth.authenticate_user()
# project_id = "phd-dissertation-343113"
# !gcloud config set project {project_id}

Mounted at /content/drive


In [3]:
df = pd.read_csv('df.csv')
df.head(10)

,id_municipio,id_municipio_tse,Name,State,Region,Population,Dose1 Applied,Dose2 Applied,Dose3 Applied,Dose4 Applied,...,Catholic,Evangelic,Pentecostal Evangelic,Umbanda/Candomble,No religion,Education 8-10y,Education 11-14y,Education 15y plus,Income (BRL),Education 8y less
0,1100015,310,Alta Floresta D'Oeste,RO,Norte,22516,0.778335,0.655001,0.103260,0.000266,...,0.642388,0.114319,0.165616,0.000000,0.121069,10.13,6.52,0.74,1310.78,81.23
1,1100023,78,Ariquemes,RO,Norte,111148,0.660795,0.527999,0.110672,0.003176,...,0.329021,0.054720,0.179976,0.000000,0.163602,15.57,10.28,1.81,1849.30,71.52
2,1100031,450,Cabixi,RO,Norte,5067,0.881389,0.737320,0.114466,0.008881,...,0.761200,0.060193,0.288534,0.000000,0.113479,9.87,5.25,0.29,1232.03,82.53
3,1100049,94,Cacoal,RO,Norte,86416,0.845156,0.635554,0.107816,0.001365,...,0.388065,0.130994,0.184804,0.000000,0.117525,13.75,11.12,1.97,1853.64,71.00
4,1100056,272,Cerejeiras,RO,Norte,16088,0.781701,0.648993,0.145699,0.001740,...,0.627797,0.100945,0.180756,0.000000,0.090627,12.36,8.83,0.64,1573.62,76.52
5,1100064,230,Colorado do Oeste,RO,Norte,15213,0.859199,0.678170,0.112798,0.001775,...,0.704792,0.058437,0.254190,0.000000,0.098666,12.94,10.23,1.32,1518.89,73.01
6,1100072,655,Corumbiara,RO,Norte,7052,0.832388,0.724759,0.155559,0.000425,...,0.823880,0.057856,0.233976,0.000000,0.106069,5.95,2.95,0.34,1188.16,89.41
7,1100080,213,Costa Marques,RO,Norte,19255,0.461646,0.368476,0.060608,0.000675,...,0.448195,0.061023,0.123448,0.000208,0.034329,9.72,6.54,1.31,1225.56,81.35
8,1100098,256,Espigão D'Oeste,RO,Norte,33009,0.691690,0.530128,0.053379,0.000515,...,0.382805,0.208004,0.129540,0.000000,0.106304,10.91,6.00,0.73,1455.66,79.78
9,1100106,19,Guajará-Mirim,RO,Norte,46930,0.634626,0.439740,0.041168,0.000384,...,0.512849,0.093394,0.132325,0.000192,0.083273,17.19,12.16,2.32,1646.55,64.65


In [4]:
df['Outer wing 18'] = df['Far-left 18'] + df['Far-right18']
df['outer wing 22'] = df['Far-left 22'] + df['Far-right 22']

In [5]:
columns_corr = ['Doses Applied', 'Doses Received',  'Dose3 Applied', 'Dose3 Received', 'Vaccination coverage', 'Polio coverage',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'PL coligation 22', 
       'Population', 'elderly(>70yo)',  'Rural pop.',   'Pop. density', 'White', 'Black', 'Brown',
       'Catholic', 'Evangelic', 'No religion', 'Education 8y less', 'Education 15y plus', 'Income (BRL)']

corr = df[columns_corr].corr(method='pearson')
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

<ipython-input-5-f626901f56d4>:10: FutureWarning: this method is deprecated in favour of `Styler.format(precision=..)`
  (corr


,Doses Applied,Doses Received,Dose3 Applied,Dose3 Received,Vaccination coverage,Polio coverage,Bolsonaro Share 18,Bolsonaro Share 22,Far-left 18,Far-right 22,PL coligation 22,Population,elderly(>70yo),Pop. density,White,Brown,Catholic,Evangelic,No religion,Education 8y less,Income (BRL)
Doses Applied,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Doses Received,0.78,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose3 Applied,0.78,0.75,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dose3 Received,0.72,0.82,0.96,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Vaccination coverage,0.33,0.38,0.34,0.35,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Polio coverage,0.27,0.31,0.28,0.28,0.90,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Bolsonaro Share 18,0.39,0.33,0.36,0.38,0.23,0.14,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Bolsonaro Share 22,0.35,0.28,0.31,0.33,0.22,0.13,0.98,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Far-left 18,0.05,-0.01,-0.01,-0.01,-0.11,-0.11,0.24,0.20,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Far-right 22,0.28,0.25,0.21,0.22,0.21,0.16,0.51,0.44,0.24,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [6]:
print(corr.round(2).to_latex())

\begin{tabular}{lrrrrrrrrrrrrrrrrrrrrr}
\toprule
{} &  Doses Applied &  Doses Received &  Dose3 Applied &  Dose3 Received &  Vaccination coverage &  Polio coverage &  Bolsonaro Share 18 &  Bolsonaro Share 22 &  Far-left 18 &  Far-right 22 &  PL coligation 22 &  Population &  elderly(>70yo) &  Pop. density &  White &  Brown &  Catholic &  Evangelic &  No religion &  Education 8y less &  Income (BRL) \\
\midrule
Doses Applied        &            NaN &             NaN &            NaN &             NaN &                   NaN &             NaN &                 NaN &                 NaN &          NaN &           NaN &               NaN &         NaN &             NaN &           NaN &    NaN &    NaN &       NaN &        NaN &          NaN &                NaN &           NaN \\
Doses Received       &           0.78 &             NaN &            NaN &             NaN &                   NaN &             NaN &                 NaN &                 NaN &          NaN &           NaN &   

<ipython-input-6-aca0328c279e>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(corr.round(2).to_latex())


In [7]:
df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Ciro Share 18', 'Far-left 22',
       'Far-right 22', 'PL coligation 22', 'Tebet Share 22',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage',
       'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous',
       'Pop. density', 'Catholic', 'Evangelic', 'Pentecostal Evangelic',
       'Umbanda/Candomble', 'No religion', 'Education 8-10y',
       'Education 11-14y', 'Education 15y plus', 'Income (BRL)',
       'Education 8y less', 'Outer wing 18', 'outer wing 22'],
      dtype='object')

In [191]:
# y_vars = 'Doses Applied', 'Doses Received', 'Dose3 Applied', 'Dose3 Received', 'Vaccination coverage', 'Polio coverage'
# x_vars1 = ('Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
# # x_vars1 = ('PSL coligation 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
# #            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
# x_vars2 = ('Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')
# x_vars3 = ('outer wing 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

# x_vars4 = ('Ciro Share 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

# x_vars5 = ('Tebet Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)')

In [192]:
y_vars = 'Doses Applied', 'Doses Received', 'Dose3 Applied', 'Dose3 Received', 'Vaccination coverage', 'Polio coverage'

x_vars1 = ('Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

x_vars2 = ('Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

x_vars3 = ('outer wing 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

x_vars4 = ('Ciro Share 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

x_vars5 = ('Tebet Share 22', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

x_vars6 = ('Bolsonaro Share 18', 'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous', 'Pop. density', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)')

In [193]:
df[list(x_vars1)] = df[list(x_vars1)].replace('-', 0).astype('float64')

In [194]:
models = []

In [195]:
# setting DF for OLS
X1 = sm.add_constant(df[list(x_vars1)])
X2 = sm.add_constant(df[list(x_vars2)])
X3 = sm.add_constant(df[list(x_vars3)])
X4 = sm.add_constant(df[list(x_vars4)])
X5 = sm.add_constant(df[list(x_vars5)])
X6 = sm.add_constant(df[list(x_vars6)])

# target1 (y1) as Doses Applied
y1 = df[y_vars[0]].values.reshape(-1, 1)

# target2 (y2) as Doses Received
y2 = df[y_vars[1]].values.reshape(-1, 1)

# target3 (y3) as Dose3 Applied
y3 = df[y_vars[2]].values.reshape(-1, 1)

# target4 (y24 as Dose3 Received
y4 = df[y_vars[3]].values.reshape(-1, 1)

# target5 (y5) as General vaxx coverage
y5 = df[y_vars[4]].values.reshape(-1, 1)

# target6 (y6) as Polio vaxx coverage
y6 = df[y_vars[5]].values.reshape(-1, 1)



In [196]:
# Doses aplicadas ~ Bolsonaro Share '22

# Create the OLS model with a different name
model1 = sm.OLS(y1, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     412.1
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                -1319.0
No. Observations:                5507   AIC:                             2668.
Df Residuals:                    5492   BIC:                             2767.
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.9988      0.460      4.349      0.000       1.098       2.900
Bolsonaro Share 22        0.0595      0.049      1.223      0.221      -0.036       0.155
elderly(>70yo)            0.0442      0.003     12.798      0.000       0.037       0.051
Rural pop.            -3.772e-05      0.000     -0.133      0.894      -0.001       0.001
White                    -0.0170      0.005     -3.778      0.000      -0.026      -0.008
Black                    -0.0171      0.005     -3.546      0.000      -0.027      -0.008
Brown                    -0.0226      0.005     -4.962      0.000      -0.032      -0.014
Indigenous               -0.0206      0.005     -4.472      0.000      -0.030      -0.012
Pop. density           -1.03e-05    1.1e-05     -0.939      0.348   -3.18e-05    1.12e-05
Catholic                  1.0296      0.073     14.165      0.000       0.887       1.172
Evangelic                 0.6775      0.107      6.328      0.000       0.468       0.887
Pentecostal Evangelic     1.1845      0.110     10.781      0.000       0.969       1.400
No religion               1.0785      0.139      7.773      0.000       0.807       1.350
Education 11-14y          0.0106      0.002      6.179      0.000       0.007       0.014
Income (BRL)              0.0001   1.63e-05      7.520      0.000    9.06e-05       0.000
==============================================================================
Omnibus:                     3276.993   Durbin-Watson:                   1.408
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           491552.810
Skew:                           1.834   Prob(JB):                         0.00
Kurtosis:                      49.139   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [197]:
# Doses recebidas ~ Bolsonaro Share '22

# Create the OLS model with a different name
model2 = sm.OLS(y2, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     393.8
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                -371.24
No. Observations:                5507   AIC:                             772.5
Df Residuals:                    5492   BIC:                             871.7
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.1919      0.420      5.224      0.000       1.370       3.014
Bolsonaro Share 22       -0.0321      0.042     -0.761      0.447      -0.115       0.051
elderly(>70yo)            0.0599      0.004     13.986      0.000       0.052       0.068
Rural pop.               -0.0013      0.000     -4.967      0.000      -0.002      -0.001
White                    -0.0191      0.004     -4.752      0.000      -0.027      -0.011
Black                    -0.0241      0.004     -5.636      0.000      -0.033      -0.016
Brown                    -0.0239      0.004     -5.886      0.000      -0.032      -0.016
Indigenous               -0.0228      0.004     -5.370      0.000      -0.031      -0.015
Pop. density          -6.612e-06   7.67e-06     -0.863      0.388   -2.16e-05    8.41e-06
Catholic                  1.2067      0.120     10.024      0.000       0.971       1.443
Evangelic                 0.7327      0.147      4.995      0.000       0.445       1.020
Pentecostal Evangelic     1.1497      0.190      6.062      0.000       0.778       1.521
No religion               1.0296      0.167      6.157      0.000       0.702       1.357
Education 11-14y          0.0089      0.002      5.283      0.000       0.006       0.012
Income (BRL)           9.935e-05   1.73e-05      5.745      0.000    6.55e-05       0.000
==============================================================================
Omnibus:                      762.876   Durbin-Watson:                   1.842
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3779.957
Skew:                           0.576   Prob(JB):                         0.00
Kurtosis:                       6.892   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [198]:
# 3ª dose aplicada ~ Bolsonaro Share '22

# Create the OLS model with a different name
model3 = sm.OLS(y3, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.440
Method:                 Least Squares   F-statistic:                     305.4
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                 7490.2
No. Observations:                5507   AIC:                        -1.495e+04
Df Residuals:                    5492   BIC:                        -1.485e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0122      0.098     -0.125      0.901      -0.204       0.180
Bolsonaro Share 22       -0.0047      0.010     -0.458      0.647      -0.025       0.015
elderly(>70yo)            0.0169      0.001     21.061      0.000       0.015       0.018
Rural pop.                0.0002   6.16e-05      3.565      0.000    9.89e-05       0.000
White                    -0.0002      0.001     -0.242      0.809      -0.002       0.002
Black                    -0.0014      0.001     -1.393      0.164      -0.003       0.001
Brown                    -0.0012      0.001     -1.276      0.202      -0.003       0.001
Indigenous               -0.0004      0.001     -0.375      0.708      -0.002       0.002
Pop. density          -2.048e-06   1.31e-06     -1.564      0.118   -4.61e-06    5.19e-07
Catholic                  0.0890      0.014      6.181      0.000       0.061       0.117
Evangelic                 0.0208      0.028      0.749      0.454      -0.034       0.075
Pentecostal Evangelic     0.1440      0.030      4.734      0.000       0.084       0.204
No religion               0.0953      0.026      3.617      0.000       0.044       0.147
Education 11-14y          0.0018      0.000      4.413      0.000       0.001       0.003
Income (BRL)           2.057e-05   4.45e-06      4.621      0.000    1.18e-05    2.93e-05
==============================================================================
Omnibus:                     2066.083   Durbin-Watson:                   1.783
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22092.969
Skew:                           1.484   Prob(JB):                         0.00
Kurtosis:                      12.353   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [199]:
# 3ª dose recebida ~ Bolsonaro Share '22

# Create the OLS model with a different name
model4 = sm.OLS(y4, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     351.3
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                 7961.6
No. Observations:                5507   AIC:                        -1.589e+04
Df Residuals:                    5492   BIC:                        -1.579e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.0016      0.090      0.018      0.985      -0.174       0.178
Bolsonaro Share 22        0.0009      0.009      0.099      0.921      -0.016       0.018
elderly(>70yo)            0.0170      0.001     22.578      0.000       0.016       0.018
Rural pop.                0.0001    5.5e-05      2.077      0.038    6.41e-06       0.000
White                    -0.0004      0.001     -0.483      0.629      -0.002       0.001
Black                    -0.0017      0.001     -1.838      0.066      -0.004       0.000
Brown                    -0.0014      0.001     -1.601      0.109      -0.003       0.000
Indigenous               -0.0005      0.001     -0.568      0.570      -0.002       0.001
Pop. density          -1.347e-06   1.27e-06     -1.059      0.289   -3.84e-06    1.15e-06
Catholic                  0.1025      0.015      6.732      0.000       0.073       0.132
Evangelic                 0.0281      0.027      1.056      0.291      -0.024       0.080
Pentecostal Evangelic     0.1402      0.031      4.470      0.000       0.079       0.202
No religion               0.0968      0.025      3.845      0.000       0.047       0.146
Education 11-14y          0.0019      0.000      5.171      0.000       0.001       0.003
Income (BRL)           1.842e-05   3.92e-06      4.704      0.000    1.07e-05    2.61e-05
==============================================================================
Omnibus:                     1541.884   Durbin-Watson:                   1.795
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10016.538
Skew:                           1.176   Prob(JB):                         0.00
Kurtosis:                       9.174   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [200]:
# Cobertura vacinal ~ Bolsonaro Share '22

# Create the OLS model with a different name
model5 = sm.OLS(y5, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     74.37
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          9.15e-195
Time:                        13:23:39   Log-Likelihood:                -24808.
No. Observations:                5507   AIC:                         4.965e+04
Df Residuals:                    5492   BIC:                         4.975e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   158.2069     33.739      4.689      0.000      92.079     224.335
Bolsonaro Share 22       22.5957      3.386      6.673      0.000      15.959      29.233
elderly(>70yo)            2.1761      0.261      8.332      0.000       1.664       2.688
Rural pop.               -0.0058      0.021     -0.280      0.780      -0.047       0.035
White                    -1.0404      0.337     -3.084      0.002      -1.702      -0.379
Black                    -1.1368      0.356     -3.193      0.001      -1.835      -0.439
Brown                    -1.1658      0.340     -3.426      0.001      -1.833      -0.499
Indigenous               -1.3337      0.343     -3.888      0.000      -2.006      -0.661
Pop. density             -0.0017      0.001     -2.639      0.008      -0.003      -0.000
Catholic                 13.0833      3.189      4.102      0.000       6.832      19.334
Evangelic                 0.2185      8.024      0.027      0.978     -15.508      15.945
Pentecostal Evangelic     8.7807      6.404      1.371      0.170      -3.771      21.332
No religion             -58.8861      9.754     -6.037      0.000     -78.003     -39.769
Education 11-14y         -0.9349      0.134     -6.980      0.000      -1.197      -0.672
Income (BRL)              0.0055      0.001      3.957      0.000       0.003       0.008
==============================================================================
Omnibus:                      567.833   Durbin-Watson:                   1.881
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2174.576
Skew:                           0.469   Prob(JB):                         0.00
Kurtosis:                       5.932   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [201]:
# Cobertura contra Polio ~ Bolsonaro share '22

# Create the OLS model with a different name
model6 = sm.OLS(y6, X1).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.119
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     43.58
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.43e-114
Time:                        13:23:39   Log-Likelihood:                -26188.
No. Observations:                5507   AIC:                         5.241e+04
Df Residuals:                    5492   BIC:                         5.251e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   184.1807     43.435      4.240      0.000      99.050     269.311
Bolsonaro Share 22       18.2789      4.521      4.043      0.000       9.418      27.140
elderly(>70yo)            2.6357      0.353      7.468      0.000       1.944       3.327
Rural pop.               -0.0026      0.026     -0.101      0.919      -0.053       0.048
White                    -1.1440      0.436     -2.625      0.009      -1.998      -0.290
Black                    -1.3142      0.458     -2.869      0.004      -2.212      -0.416
Brown                    -1.2945      0.439     -2.951      0.003      -2.154      -0.435
Indigenous               -1.5824      0.439     -3.602      0.000      -2.443      -0.721
Pop. density             -0.0014      0.001     -1.802      0.072      -0.003       0.000
Catholic                 13.4775      4.017      3.355      0.001       5.604      21.351
Evangelic                -8.6826     13.133     -0.661      0.509     -34.423      17.058
Pentecostal Evangelic     6.1625      8.441      0.730      0.465     -10.382      22.707
No religion             -46.8614     12.520     -3.743      0.000     -71.401     -22.322
Education 11-14y         -1.0174      0.172     -5.920      0.000      -1.354      -0.681
Income (BRL)              0.0036      0.002      2.002      0.045    7.69e-05       0.007
==============================================================================
Omnibus:                     2173.436   Durbin-Watson:                   1.937
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34397.755
Skew:                           1.459   Prob(JB):                         0.00
Kurtosis:                      14.891   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [202]:
# Doses aplicadas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model1b = sm.OLS(y1, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1b)

model1b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.450
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     388.2
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                -1317.8
No. Observations:                5507   AIC:                             2668.
Df Residuals:                    5491   BIC:                             2773.
Df Model:                          15                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.0790      0.457      4.549      0.000       1.183       2.975
Far-left 18               3.8124      2.015      1.892      0.058      -0.136       7.761
Far-right 22              0.5704      1.277      0.447      0.655      -1.933       3.073
elderly(>70yo)            0.0445      0.003     12.837      0.000       0.038       0.051
Rural pop.            -2.956e-05      0.000     -0.103      0.918      -0.001       0.001
White                    -0.0177      0.004     -3.937      0.000      -0.027      -0.009
Black                    -0.0183      0.005     -3.816      0.000      -0.028      -0.009
Brown                    -0.0234      0.005     -5.157      0.000      -0.032      -0.015
Indigenous               -0.0215      0.005     -4.679      0.000      -0.031      -0.013
Pop. density          -1.589e-05   1.11e-05     -1.430      0.153   -3.77e-05    5.89e-06
Catholic                  1.0241      0.073     13.941      0.000       0.880       1.168
Evangelic                 0.6969      0.107      6.532      0.000       0.488       0.906
Pentecostal Evangelic     1.2328      0.107     11.575      0.000       1.024       1.442
No religion               1.0688      0.141      7.587      0.000       0.793       1.345
Education 11-14y          0.0102      0.002      5.862      0.000       0.007       0.014
Income (BRL)              0.0001   1.53e-05      8.391      0.000    9.84e-05       0.000
==============================================================================
Omnibus:                     3275.216   Durbin-Watson:                   1.406
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           492803.539
Skew:                           1.831   Prob(JB):                         0.00
Kurtosis:                      49.198   Cond. No.                     7.26e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.26e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [203]:
# Doses recebidas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2b = sm.OLS(y2, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2b)

model2b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     367.4
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                -368.34
No. Observations:                5507   AIC:                             768.7
Df Residuals:                    5491   BIC:                             874.5
Df Model:                          15                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.1239      0.419      5.071      0.000       1.303       2.945
Far-left 18               2.2513      1.743      1.292      0.197      -1.165       5.668
Far-right 22             -2.9119      1.252     -2.326      0.020      -5.366      -0.458
elderly(>70yo)            0.0609      0.004     14.512      0.000       0.053       0.069
Rural pop.               -0.0014      0.000     -5.135      0.000      -0.002      -0.001
White                    -0.0185      0.004     -4.628      0.000      -0.026      -0.011
Black                    -0.0233      0.004     -5.439      0.000      -0.032      -0.015
Brown                    -0.0234      0.004     -5.764      0.000      -0.031      -0.015
Indigenous               -0.0224      0.004     -5.293      0.000      -0.031      -0.014
Pop. density          -6.663e-06   7.36e-06     -0.905      0.366   -2.11e-05    7.77e-06
Catholic                  1.2115      0.121     10.034      0.000       0.975       1.448
Evangelic                 0.7258      0.143      5.069      0.000       0.445       1.006
Pentecostal Evangelic     1.1241      0.175      6.407      0.000       0.780       1.468
No religion               0.9892      0.168      5.877      0.000       0.659       1.319
Education 11-14y          0.0085      0.002      4.991      0.000       0.005       0.012
Income (BRL)           9.972e-05   1.53e-05      6.521      0.000    6.98e-05       0.000
==============================================================================
Omnibus:                      752.012   Durbin-Watson:                   1.839
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3642.800
Skew:                           0.573   Prob(JB):                         0.00
Kurtosis:                       6.816   Cond. No.                     7.26e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.26e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [204]:
# 3ª dose aplicada ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model3b = sm.OLS(y3, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3b)

model3b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.445
Method:                 Least Squares   F-statistic:                     287.8
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                 7513.6
No. Observations:                5507   AIC:                        -1.500e+04
Df Residuals:                    5491   BIC:                        -1.489e+04
Df Model:                          15                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0619      0.098     -0.633      0.527      -0.254       0.130
Far-left 18              -0.0785      0.447     -0.176      0.861      -0.955       0.798
Far-right 22             -2.0162      0.337     -5.982      0.000      -2.677      -1.356
elderly(>70yo)            0.0170      0.001     21.700      0.000       0.015       0.019
Rural pop.                0.0002   6.12e-05      3.128      0.002    7.15e-05       0.000
White                     0.0003      0.001      0.334      0.738      -0.002       0.002
Black                    -0.0007      0.001     -0.640      0.522      -0.003       0.001
Brown                    -0.0008      0.001     -0.768      0.442      -0.003       0.001
Indigenous                0.0001      0.001      0.132      0.895      -0.002       0.002
Pop. density          -1.149e-06    1.3e-06     -0.884      0.376    -3.7e-06     1.4e-06
Catholic                  0.0897      0.014      6.247      0.000       0.062       0.118
Evangelic                 0.0194      0.027      0.728      0.466      -0.033       0.072
Pentecostal Evangelic     0.1345      0.027      4.899      0.000       0.081       0.188
No religion               0.0733      0.027      2.765      0.006       0.021       0.125
Education 11-14y          0.0018      0.000      4.130      0.000       0.001       0.003
Income (BRL)           2.435e-05   3.98e-06      6.118      0.000    1.65e-05    3.21e-05
==============================================================================
Omnibus:                     2070.882   Durbin-Watson:                   1.783
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21911.300
Skew:                           1.492   Prob(JB):                         0.00
Kurtosis:                      12.305   Cond. No.                     7.26e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.26e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [205]:
# 3ª dose recebida ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model4b = sm.OLS(y4, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4b)

model4b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.500
Method:                 Least Squares   F-statistic:                     330.9
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:39   Log-Likelihood:                 7989.4
No. Observations:                5507   AIC:                        -1.595e+04
Df Residuals:                    5491   BIC:                        -1.584e+04
Df Model:                          15                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0445      0.089     -0.497      0.619      -0.220       0.131
Far-left 18              -0.0452      0.398     -0.114      0.909      -0.825       0.734
Far-right 22             -2.0150      0.310     -6.498      0.000      -2.623      -1.407
elderly(>70yo)            0.0171      0.001     23.066      0.000       0.016       0.019
Rural pop.             8.636e-05   5.47e-05      1.580      0.114   -2.08e-05       0.000
White                     0.0001      0.001      0.131      0.895      -0.002       0.002
Black                    -0.0010      0.001     -1.063      0.288      -0.003       0.001
Brown                    -0.0010      0.001     -1.086      0.277      -0.003       0.001
Indigenous            -4.447e-05      0.001     -0.049      0.961      -0.002       0.002
Pop. density          -6.566e-07   1.27e-06     -0.516      0.606   -3.15e-06    1.84e-06
Catholic                  0.1027      0.015      6.745      0.000       0.073       0.133
Evangelic                 0.0282      0.026      1.099      0.272      -0.022       0.079
Pentecostal Evangelic     0.1344      0.028      4.718      0.000       0.079       0.190
No religion               0.0746      0.025      2.945      0.003       0.025       0.124
Education 11-14y          0.0018      0.000      4.841      0.000       0.001       0.003
Income (BRL)           2.302e-05   3.58e-06      6.437      0.000     1.6e-05       3e-05
==============================================================================
Omnibus:                     1544.027   Durbin-Watson:                   1.795
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9912.378
Skew:                           1.182   Prob(JB):                         0.00
Kurtosis:                       9.133   Cond. No.                     7.26e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.26e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [206]:
# Cobertura vacinal geral ~ Far-left '18, Far-right '22

# Create the OLS model with a different name
model5b = sm.OLS(y5, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5b)

model5b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.180
Method:                 Least Squares   F-statistic:                     72.38
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          7.58e-202
Time:                        13:23:39   Log-Likelihood:                -24803.
No. Observations:                5507   AIC:                         4.964e+04
Df Residuals:                    5491   BIC:                         4.974e+04
Df Model:                          15                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   179.9617     33.735      5.335      0.000     113.843     246.080
Far-left 18            -817.2533    136.140     -6.003      0.000   -1084.084    -550.423
Far-right 22            639.8686    111.148      5.757      0.000     422.022     857.715
elderly(>70yo)            1.7424      0.258      6.757      0.000       1.237       2.248
Rural pop.                0.0070      0.021      0.336      0.737      -0.034       0.048
White                    -1.1488      0.338     -3.399      0.001      -1.811      -0.486
Black                    -1.3733      0.358     -3.836      0.000      -2.075      -0.672
Brown                    -1.3261      0.340     -3.897      0.000      -1.993      -0.659
Indigenous               -1.4484      0.343     -4.226      0.000      -2.120      -0.777
Pop. density             -0.0019      0.001     -2.769      0.006      -0.003      -0.001
Catholic                 10.1272      3.186      3.179      0.001       3.883      16.371
Evangelic                 5.6694      7.904      0.717      0.473      -9.823      21.162
Pentecostal Evangelic    23.9883      6.281      3.819      0.000      11.677      36.299
No religion             -49.0800      9.661     -5.080      0.000     -68.016     -30.144
Education 11-14y         -0.8010      0.135     -5.932      0.000      -1.066      -0.536
Income (BRL)              0.0080      0.001      5.827      0.000       0.005       0.011
==============================================================================
Omnibus:                      538.643   Durbin-Watson:                   1.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1983.683
Skew:                           0.452   Prob(JB):                         0.00
Kurtosis:                       5.798   Cond. No.                     7.26e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.26e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [207]:
# Cobertura contra Polio ~ Far-left '18, Far-right '22

# Create the OLS model with a different name
model6b = sm.OLS(y6, X2).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6b)

model6b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.123
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     43.06
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.39e-120
Time:                        13:23:39   Log-Likelihood:                -26174.
No. Observations:                5507   AIC:                         5.238e+04
Df Residuals:                    5491   BIC:                         5.249e+04
Df Model:                          15                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   205.4204     43.474      4.725      0.000     120.214     290.627
Far-left 18            -883.0715    171.993     -5.134      0.000   -1220.171    -545.972
Far-right 22            754.8963    152.703      4.944      0.000     455.605    1054.188
elderly(>70yo)            2.2221      0.343      6.477      0.000       1.550       2.894
Rural pop.                0.0115      0.026      0.443      0.658      -0.039       0.062
White                    -1.2691      0.437     -2.906      0.004      -2.125      -0.413
Black                    -1.5548      0.460     -3.377      0.001      -2.457      -0.652
Brown                    -1.4537      0.439     -3.310      0.001      -2.314      -0.593
Indigenous               -1.6959      0.440     -3.855      0.000      -2.558      -0.834
Pop. density             -0.0014      0.001     -1.784      0.074      -0.003       0.000
Catholic                 10.9752      3.988      2.752      0.006       3.159      18.791
Evangelic                -4.4536     12.888     -0.346      0.730     -29.714      20.807
Pentecostal Evangelic    18.7611      8.253      2.273      0.023       2.585      34.937
No religion             -35.4351     12.352     -2.869      0.004     -59.645     -11.225
Education 11-14y         -0.8720      0.172     -5.075      0.000      -1.209      -0.535
Income (BRL)              0.0052      0.002      2.806      0.005       0.002       0.009
==============================================================================
Omnibus:                     2147.030   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33168.902
Skew:                           1.443   Prob(JB):                         0.00
Kurtosis:                      14.672   Cond. No.                     7.26e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 7.26e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [208]:
models

In [209]:
# table1 = Stargazer([model1, model2, model3, model4, model5, model6])
# table1.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])
# table2 = Stargazer([model1b, model2b, model3b, model4b, model5b, model6b ])
# table2.covariate_order(['Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
#            'Pentecostal Evangelic', 'No religion', 'Education 8y less','Education 8-10y', 'Education 11-14y', 'Education 15y plus','Income (BRL)'])

In [210]:
table1 = Stargazer([model1, model2, model3, model4, model5, model6])
table1.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table1.show_degrees_of_freedom(False)
table2 = Stargazer([model1b, model2b, model3b, model4b, model5b, model6b ])
table2.covariate_order(['Far-left 18', 'Far-right 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table2.show_degrees_of_freedom(False)

In [211]:
print(table1.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & 0.059$^{}$ & -0.032$^{}$ & -0.005$^{}$ & 0.001$^{}$ & 22.596$^{***}$ & 18.279$^{***}$ \\
  & (0.049) & (0.042) & (0.010) & (0.009) & (3.386) & (4.521) \\
 elderly(>70yo) & 0.044$^{***}$ & 0.060$^{***}$ & 0.017$^{***}$ & 0.017$^{***}$ & 2.176$^{***}$ & 2.636$^{***}$ \\
  & (0.003) & (0.004) & (0.001) & (0.001) & (0.261) & (0.353) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{**}$ & -0.006$^{}$ & -0.003$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.001$^{*}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.017$^{***}$ & -0.019$^{***}$ & -0.000$^{}$ 

In [212]:
print(table2.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Far-left 18 & 3.812$^{*}$ & 2.251$^{}$ & -0.078$^{}$ & -0.045$^{}$ & -817.253$^{***}$ & -883.071$^{***}$ \\
  & (2.015) & (1.743) & (0.447) & (0.398) & (136.140) & (171.993) \\
 Far-right 22 & 0.570$^{}$ & -2.912$^{**}$ & -2.016$^{***}$ & -2.015$^{***}$ & 639.869$^{***}$ & 754.896$^{***}$ \\
  & (1.277) & (1.252) & (0.337) & (0.310) & (111.148) & (152.703) \\
 elderly(>70yo) & 0.044$^{***}$ & 0.061$^{***}$ & 0.017$^{***}$ & 0.017$^{***}$ & 1.742$^{***}$ & 2.222$^{***}$ \\
  & (0.003) & (0.004) & (0.001) & (0.001) & (0.258) & (0.343) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{}$ & 0.007$^{}$ & 0.011$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ 

In [213]:
df['Region'].unique()

array(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'],
      dtype=object)

In [214]:
# setting DF for OLS
df_co = df[df['Region'] == 'Centro-Oeste']
df_ne = df[df['Region'] == 'Nordeste']
df_no = df[df['Region'] == 'Norte']
df_se = df[df['Region'] == 'Sudeste']
df_su = df[df['Region'] == 'Sul']

X2_co = sm.add_constant(df_co[list(x_vars1)])
X2_ne = sm.add_constant(df_ne[list(x_vars1)])
X2_no = sm.add_constant(df_no[list(x_vars1)])
X2_se = sm.add_constant(df_se[list(x_vars1)])
X2_su = sm.add_constant(df_su[list(x_vars1)])

# target2 (y2) as Doses Received
y2_co = df_co[y_vars[1]].values.reshape(-1, 1)
y2_ne = df_ne[y_vars[1]].values.reshape(-1, 1)
y2_no = df_no[y_vars[1]].values.reshape(-1, 1)
y2_se = df_se[y_vars[1]].values.reshape(-1, 1)
y2_su = df_su[y_vars[1]].values.reshape(-1, 1)

# target5 (y5) as General vaxx coverage
y5_co = df_co[y_vars[4]].values.reshape(-1, 1)
y5_ne = df_ne[y_vars[4]].values.reshape(-1, 1)
y5_no = df_no[y_vars[4]].values.reshape(-1, 1)
y5_se = df_se[y_vars[4]].values.reshape(-1, 1)
y5_su = df_su[y_vars[4]].values.reshape(-1, 1)

In [215]:
# Doses recebidas ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2co = sm.OLS(y2_co, X2_co).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2co)

model2co.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.596
Method:                 Least Squares   F-statistic:                     57.11
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           7.58e-89
Time:                        13:23:40   Log-Likelihood:                -16.775
No. Observations:                 446   AIC:                             63.55
Df Residuals:                     431   BIC:                             125.1
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     3.3285      1.428      2.330      0.020       0.529       6.128
Bolsonaro Share 22       -0.5999      0.159     -3.776      0.000      -0.911      -0.288
elderly(>70yo)            0.0124      0.012      0.993      0.321      -0.012       0.037
Rural pop.               -0.0005      0.001     -0.461      0.644      -0.003       0.002
White                    -0.0254      0.015     -1.723      0.085      -0.054       0.004
Black                    -0.0524      0.016     -3.292      0.001      -0.084      -0.021
Brown                    -0.0334      0.015     -2.291      0.022      -0.062      -0.005
Indigenous               -0.0326      0.015     -2.246      0.025      -0.061      -0.004
Pop. density             -0.0002   6.69e-05     -2.557      0.011      -0.000   -3.99e-05
Catholic                  1.4069      0.101     13.923      0.000       1.209       1.605
Evangelic                -0.5200      0.508     -1.024      0.306      -1.515       0.475
Pentecostal Evangelic     1.8478      0.169     10.925      0.000       1.516       2.179
No religion               0.9443      0.348      2.712      0.007       0.262       1.627
Education 11-14y          0.0153      0.005      2.963      0.003       0.005       0.025
Income (BRL)              0.0002   6.78e-05      2.926      0.003    6.55e-05       0.000
==============================================================================
Omnibus:                       40.110   Durbin-Watson:                   2.128
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.641
Skew:                           0.623   Prob(JB):                     6.79e-14
Kurtosis:                       4.308   Cond. No.                     1.88e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.88e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [216]:
# Create the OLS model with a different name
model2ne = sm.OLS(y2_ne, X2_ne).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2ne)

model2ne.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.483
Model:                            OLS   Adj. R-squared:                  0.478
Method:                 Least Squares   F-statistic:                     76.77
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          7.61e-171
Time:                        13:23:40   Log-Likelihood:                -49.374
No. Observations:                1787   AIC:                             128.7
Df Residuals:                    1772   BIC:                             211.1
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.6763      0.579      2.896      0.004       0.542       2.811
Bolsonaro Share 22       -0.3812      0.092     -4.158      0.000      -0.561      -0.202
elderly(>70yo)            0.0613      0.007      8.832      0.000       0.048       0.075
Rural pop.               -0.0013      0.000     -3.273      0.001      -0.002      -0.001
White                    -0.0165      0.005     -3.040      0.002      -0.027      -0.006
Black                    -0.0196      0.006     -3.372      0.001      -0.031      -0.008
Brown                    -0.0226      0.006     -4.104      0.000      -0.033      -0.012
Indigenous               -0.0180      0.006     -3.178      0.001      -0.029      -0.007
Pop. density          -2.458e-05   9.48e-06     -2.593      0.010   -4.32e-05      -6e-06
Catholic                  1.5032      0.225      6.685      0.000       1.062       1.944
Evangelic                -0.0845      0.427     -0.198      0.843      -0.922       0.753
Pentecostal Evangelic     1.0458      0.270      3.879      0.000       0.517       1.574
No religion               1.5547      0.242      6.435      0.000       1.081       2.028
Education 11-14y          0.0068      0.003      1.991      0.046       0.000       0.013
Income (BRL)              0.0003   4.13e-05      7.025      0.000       0.000       0.000
==============================================================================
Omnibus:                      184.145   Durbin-Watson:                   1.871
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1003.221
Skew:                           0.316   Prob(JB):                    1.42e-218
Kurtosis:                       6.616   Cond. No.                     9.04e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 9.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [217]:
# Create the OLS model with a different name
model2no = sm.OLS(y2_no, X2_no).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2no)

model2no.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.546
Method:                 Least Squares   F-statistic:                     43.51
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           1.45e-73
Time:                        13:23:40   Log-Likelihood:                 19.372
No. Observations:                 449   AIC:                            -8.743
Df Residuals:                     434   BIC:                             52.86
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.1668      1.117      0.149      0.881      -2.023       2.356
Bolsonaro Share 22       -0.0595      0.099     -0.602      0.547      -0.253       0.134
elderly(>70yo)            0.0939      0.014      6.808      0.000       0.067       0.121
Rural pop.                0.0012      0.001      1.360      0.174      -0.001       0.003
White                     0.0053      0.011      0.481      0.631      -0.016       0.027
Black                    -0.0098      0.012     -0.813      0.416      -0.033       0.014
Brown                    -0.0057      0.011     -0.519      0.603      -0.027       0.016
Indigenous               -0.0033      0.011     -0.299      0.765      -0.025       0.019
Pop. density          -3.533e-05   4.11e-05     -0.859      0.391      -0.000    4.53e-05
Catholic                  0.8797      0.104      8.484      0.000       0.676       1.083
Evangelic                 0.0564      0.470      0.120      0.904      -0.864       0.977
Pentecostal Evangelic     0.2439      0.204      1.196      0.232      -0.156       0.644
No religion               0.7010      0.418      1.677      0.094      -0.118       1.520
Education 11-14y          0.0131      0.006      2.227      0.026       0.002       0.025
Income (BRL)              0.0003   6.76e-05      3.866      0.000       0.000       0.000
==============================================================================
Omnibus:                       89.302   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              311.276
Skew:                           0.872   Prob(JB):                     2.55e-68
Kurtosis:                       6.688   Cond. No.                     1.39e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.39e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [218]:
# Create the OLS model with a different name
model2se = sm.OLS(y2_se, X2_se).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2se)

model2se.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     62.49
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          2.85e-141
Time:                        13:23:40   Log-Likelihood:                -118.08
No. Observations:                1666   AIC:                             266.2
Df Residuals:                    1651   BIC:                             347.4
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.9312      0.920      2.098      0.036       0.127       3.735
Bolsonaro Share 22        0.0507      0.089      0.571      0.568      -0.123       0.225
elderly(>70yo)            0.0435      0.006      7.842      0.000       0.033       0.054
Rural pop.               -0.0008      0.001     -1.468      0.142      -0.002       0.000
White                    -0.0130      0.009     -1.439      0.150      -0.031       0.005
Black                    -0.0228      0.010     -2.389      0.017      -0.042      -0.004
Brown                    -0.0182      0.009     -1.990      0.047      -0.036      -0.000
Indigenous               -0.0177      0.009     -1.923      0.054      -0.036       0.000
Pop. density           1.452e-06   8.34e-06      0.174      0.862   -1.49e-05    1.78e-05
Catholic                  1.0961      0.116      9.434      0.000       0.868       1.324
Evangelic                 0.7850      0.199      3.946      0.000       0.395       1.175
Pentecostal Evangelic     1.1701      0.185      6.336      0.000       0.808       1.532
No religion               0.7530      0.298      2.527      0.012       0.169       1.337
Education 11-14y          0.0047      0.003      1.487      0.137      -0.002       0.011
Income (BRL)            4.97e-05   2.62e-05      1.898      0.058   -1.63e-06       0.000
==============================================================================
Omnibus:                      226.285   Durbin-Watson:                   1.943
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              516.105
Skew:                           0.781   Prob(JB):                    8.50e-113
Kurtosis:                       5.235   Cond. No.                     2.14e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.14e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [219]:
# Create the OLS model with a different name
model2su = sm.OLS(y2_su, X2_su).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2su)

model2su.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.541
Model:                            OLS   Adj. R-squared:                  0.535
Method:                 Least Squares   F-statistic:                     58.90
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          3.95e-124
Time:                        13:23:40   Log-Likelihood:                 147.34
No. Observations:                1159   AIC:                            -264.7
Df Residuals:                    1144   BIC:                            -188.9
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0376      0.880     -0.043      0.966      -1.763       1.688
Bolsonaro Share 22       -0.0409      0.080     -0.513      0.608      -0.197       0.115
elderly(>70yo)            0.0369      0.005      6.736      0.000       0.026       0.048
Rural pop.               -0.0003      0.001     -0.560      0.575      -0.001       0.001
White                     0.0013      0.009      0.151      0.880      -0.016       0.018
Black                     0.0047      0.009      0.504      0.614      -0.014       0.023
Brown                    -0.0025      0.009     -0.290      0.772      -0.020       0.015
Indigenous               -0.0016      0.009     -0.174      0.862      -0.019       0.016
Pop. density          -1.768e-05   1.79e-05     -0.990      0.322   -5.27e-05    1.73e-05
Catholic                  1.4150      0.103     13.697      0.000       1.213       1.617
Evangelic                 1.2942      0.132      9.809      0.000       1.036       1.553
Pentecostal Evangelic     1.5315      0.205      7.485      0.000       1.130       1.933
No religion               1.0781      0.445      2.423      0.015       0.206       1.950
Education 11-14y          0.0049      0.003      1.705      0.088      -0.001       0.011
Income (BRL)              0.0001   2.41e-05      4.779      0.000    6.79e-05       0.000
==============================================================================
Omnibus:                       57.144   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               92.081
Skew:                           0.397   Prob(JB):                     1.01e-20
Kurtosis:                       4.129   Cond. No.                     2.80e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.8e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [220]:
table3 = Stargazer([model2co, model2ne, model2no, model2se, model2su ])
table3.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table3.show_degrees_of_freedom(False)
print(table3.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable:}} \
\cr \cline{5-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & -0.600$^{***}$ & -0.381$^{***}$ & -0.060$^{}$ & 0.051$^{}$ & -0.041$^{}$ \\
  & (0.159) & (0.092) & (0.099) & (0.089) & (0.080) \\
 elderly(>70yo) & 0.012$^{}$ & 0.061$^{***}$ & 0.094$^{***}$ & 0.044$^{***}$ & 0.037$^{***}$ \\
  & (0.012) & (0.007) & (0.014) & (0.006) & (0.005) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.001$^{}$ & -0.001$^{}$ & -0.000$^{}$ \\
  & (0.001) & (0.000) & (0.001) & (0.001) & (0.001) \\
 Pop. density & -0.000$^{**}$ & -0.000$^{***}$ & -0.000$^{}$ & 0.000$^{}$ & -0.000$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.000) \\
 White & -0.025$^{*}$ & -0.016$^{***}$ & 0.005$^{}$ & -0.013$^{}$ & 0.001$^{}$ \\
  & (0.015) & (0.005) & (0.011) & (0.009) & (0.009) \\
 Black & -0.052$^{***}$ & -0.

In [221]:
# Vaxx coverage ~ Far-left 18, Far-right '22

# Create the OLS model with a different name
model2co = sm.OLS(y5_co, X2_co).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2co)

model2co.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     6.318
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           1.45e-11
Time:                        13:23:40   Log-Likelihood:                -2009.0
No. Observations:                 446   AIC:                             4048.
Df Residuals:                     431   BIC:                             4109.
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    59.5220    115.768      0.514      0.607    -167.380     286.424
Bolsonaro Share 22       24.0919     15.482      1.556      0.120      -6.252      54.436
elderly(>70yo)            1.0711      1.029      1.041      0.298      -0.946       3.088
Rural pop.                0.1272      0.081      1.562      0.118      -0.032       0.287
White                     0.1257      1.181      0.106      0.915      -2.189       2.441
Black                    -0.7040      1.345     -0.523      0.601      -3.340       1.932
Brown                    -0.0421      1.150     -0.037      0.971      -2.296       2.212
Indigenous               -0.4489      1.145     -0.392      0.695      -2.694       1.796
Pop. density             -0.0042      0.003     -1.504      0.133      -0.010       0.001
Catholic                 11.7847      8.834      1.334      0.182      -5.529      29.098
Evangelic               -37.8881     50.485     -0.750      0.453    -136.836      61.060
Pentecostal Evangelic    -4.0016     17.007     -0.235      0.814     -37.335      29.332
No religion              22.5674     31.019      0.728      0.467     -38.228      83.363
Education 11-14y         -0.1048      0.491     -0.213      0.831      -1.067       0.857
Income (BRL)             -0.0029      0.004     -0.673      0.501      -0.011       0.006
==============================================================================
Omnibus:                       41.054   Durbin-Watson:                   2.034
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.648
Skew:                           0.550   Prob(JB):                     8.35e-18
Kurtosis:                       4.739   Cond. No.                     1.88e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.88e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [222]:
# Create the OLS model with a different name
model2ne = sm.OLS(y5_ne, X2_ne).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2ne)

model2ne.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     19.95
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           2.17e-47
Time:                        13:23:40   Log-Likelihood:                -7942.3
No. Observations:                1787   AIC:                         1.591e+04
Df Residuals:                    1772   BIC:                         1.600e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   255.5452     48.509      5.268      0.000     160.470     350.620
Bolsonaro Share 22      -13.8283      8.137     -1.700      0.089     -29.776       2.119
elderly(>70yo)            2.2531      0.444      5.071      0.000       1.382       3.124
Rural pop.                0.0084      0.035      0.237      0.813      -0.061       0.078
White                    -1.8878      0.486     -3.887      0.000      -2.840      -0.936
Black                    -2.2609      0.522     -4.329      0.000      -3.285      -1.237
Brown                    -2.1586      0.488     -4.421      0.000      -3.116      -1.202
Indigenous               -2.3872      0.499     -4.779      0.000      -3.366      -1.408
Pop. density             -0.0026      0.002     -1.722      0.085      -0.006       0.000
Catholic                  8.2494      5.340      1.545      0.122      -2.216      18.715
Evangelic                15.1656     31.906      0.475      0.635     -47.370      77.701
Pentecostal Evangelic   -16.7300     15.628     -1.071      0.284     -47.360      13.900
No religion             -11.6302     16.001     -0.727      0.467     -42.991      19.731
Education 11-14y         -0.6954      0.269     -2.582      0.010      -1.223      -0.168
Income (BRL)              0.0106      0.005      2.298      0.022       0.002       0.020
==============================================================================
Omnibus:                      226.208   Durbin-Watson:                   1.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              776.130
Skew:                           0.610   Prob(JB):                    2.92e-169
Kurtosis:                       5.989   Cond. No.                     9.04e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 9.04e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [223]:
# Create the OLS model with a different name
model2no = sm.OLS(y5_no, X2_no).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2no)

model2no.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.234
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     12.02
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           1.07e-23
Time:                        13:23:40   Log-Likelihood:                -1973.5
No. Observations:                 449   AIC:                             3977.
Df Residuals:                     434   BIC:                             4039.
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   240.2904    117.643      2.043      0.041       9.715     470.866
Bolsonaro Share 22       -1.3674      8.876     -0.154      0.878     -18.765      16.030
elderly(>70yo)            4.9219      1.287      3.824      0.000       2.399       7.445
Rural pop.               -0.1586      0.067     -2.367      0.018      -0.290      -0.027
White                    -1.4668      1.186     -1.237      0.216      -3.791       0.858
Black                    -1.7261      1.220     -1.415      0.157      -4.117       0.665
Brown                    -2.0253      1.164     -1.740      0.082      -4.306       0.256
Indigenous               -2.0847      1.169     -1.784      0.074      -4.375       0.206
Pop. density             -0.0173      0.004     -4.646      0.000      -0.025      -0.010
Catholic                  9.3966      7.232      1.299      0.194      -4.778      23.571
Evangelic                -8.4233     32.294     -0.261      0.794     -71.718      54.871
Pentecostal Evangelic    -0.0831     15.299     -0.005      0.996     -30.068      29.902
No religion             -22.4606     23.867     -0.941      0.347     -69.240      24.319
Education 11-14y         -0.5351      0.515     -1.040      0.299      -1.544       0.474
Income (BRL)              0.0046      0.006      0.822      0.411      -0.006       0.016
==============================================================================
Omnibus:                        1.228   Durbin-Watson:                   1.832
Prob(Omnibus):                  0.541   Jarque-Bera (JB):                1.016
Skew:                           0.062   Prob(JB):                        0.602
Kurtosis:                       3.197   Cond. No.                     1.39e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.39e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [224]:
# Create the OLS model with a different name
model2se = sm.OLS(y5_se, X2_se).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2se)

model2se.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.136
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     17.37
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           1.18e-40
Time:                        13:23:40   Log-Likelihood:                -7526.5
No. Observations:                1666   AIC:                         1.508e+04
Df Residuals:                    1651   BIC:                         1.516e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   182.2339     62.137      2.933      0.003      60.447     304.021
Bolsonaro Share 22        9.0249      8.981      1.005      0.315      -8.578      26.628
elderly(>70yo)            1.6203      0.514      3.150      0.002       0.612       2.628
Rural pop.               -0.0494      0.047     -1.057      0.291      -0.141       0.042
White                    -0.8882      0.609     -1.457      0.145      -2.083       0.306
Black                    -1.1656      0.628     -1.855      0.064      -2.397       0.066
Brown                    -1.0537      0.618     -1.706      0.088      -2.264       0.157
Indigenous               -0.9038      0.616     -1.467      0.142      -2.111       0.304
Pop. density             -0.0003      0.001     -0.370      0.711      -0.002       0.001
Catholic                  3.9622      8.656      0.458      0.647     -13.003      20.928
Evangelic                19.1188     15.414      1.240      0.215     -11.092      49.329
Pentecostal Evangelic     6.4026     15.371      0.417      0.677     -23.723      36.529
No religion            -118.8025     29.322     -4.052      0.000    -176.272     -61.333
Education 11-14y         -1.3418      0.234     -5.731      0.000      -1.801      -0.883
Income (BRL)             -0.0008      0.002     -0.389      0.698      -0.005       0.003
==============================================================================
Omnibus:                       80.872   Durbin-Watson:                   1.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              266.730
Skew:                           0.097   Prob(JB):                     1.20e-58
Kurtosis:                       4.951   Cond. No.                     2.14e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.14e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [225]:
# Create the OLS model with a different name
model2su = sm.OLS(y5_su, X2_su).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2su)

model2su.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.165
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     12.22
Date:                Tue, 04 Apr 2023   Prob (F-statistic):           6.11e-27
Time:                        13:23:40   Log-Likelihood:                -5195.9
No. Observations:                1159   AIC:                         1.042e+04
Df Residuals:                    1144   BIC:                         1.050e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   -43.1134     84.123     -0.513      0.608    -207.991     121.764
Bolsonaro Share 22       -1.3549      8.722     -0.155      0.877     -18.449      15.739
elderly(>70yo)            1.5980      0.552      2.894      0.004       0.516       2.680
Rural pop.                0.1388      0.046      2.987      0.003       0.048       0.230
White                     1.0522      0.834      1.261      0.207      -0.583       2.687
Black                     0.5355      0.934      0.573      0.567      -1.296       2.367
Brown                     1.0211      0.862      1.185      0.236      -0.668       2.710
Indigenous                0.6970      0.877      0.795      0.427      -1.021       2.415
Pop. density             -0.0048      0.002     -2.712      0.007      -0.008      -0.001
Catholic                  2.8601      8.240      0.347      0.729     -13.290      19.011
Evangelic               -12.1836     13.783     -0.884      0.377     -39.197      14.830
Pentecostal Evangelic    -0.8787     16.162     -0.054      0.957     -32.556      30.799
No religion            -106.6365     38.627     -2.761      0.006    -182.344     -30.929
Education 11-14y         -0.3557      0.296     -1.202      0.229      -0.936       0.224
Income (BRL)              0.0103      0.003      3.695      0.000       0.005       0.016
==============================================================================
Omnibus:                      260.411   Durbin-Watson:                   1.985
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1985.366
Skew:                           0.820   Prob(JB):                         0.00
Kurtosis:                       9.199   Cond. No.                     2.80e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.8e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [226]:
table4 = Stargazer([model2co, model2ne, model2no, model2se, model2su])
table4.covariate_order(['Bolsonaro Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table4.show_degrees_of_freedom(False)
print(table4.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{5}{c}{\textit{Dependent variable:}} \
\cr \cline{5-6}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) \\
\hline \\[-1.8ex]
 Bolsonaro Share 22 & 24.092$^{}$ & -13.828$^{*}$ & -1.367$^{}$ & 9.025$^{}$ & -1.355$^{}$ \\
  & (15.482) & (8.137) & (8.876) & (8.981) & (8.722) \\
 elderly(>70yo) & 1.071$^{}$ & 2.253$^{***}$ & 4.922$^{***}$ & 1.620$^{***}$ & 1.598$^{***}$ \\
  & (1.029) & (0.444) & (1.287) & (0.514) & (0.552) \\
 Rural pop. & 0.127$^{}$ & 0.008$^{}$ & -0.159$^{**}$ & -0.049$^{}$ & 0.139$^{***}$ \\
  & (0.081) & (0.035) & (0.067) & (0.047) & (0.046) \\
 Pop. density & -0.004$^{}$ & -0.003$^{*}$ & -0.017$^{***}$ & -0.000$^{}$ & -0.005$^{***}$ \\
  & (0.003) & (0.002) & (0.004) & (0.001) & (0.002) \\
 White & 0.126$^{}$ & -1.888$^{***}$ & -1.467$^{}$ & -0.888$^{}$ & 1.052$^{}$ \\
  & (1.181) & (0.486) & (1.186) & (0.609) & (0.834) \\
 Black & -0.704$^{}$ & -2.261$

------------------------
Sensitivity tests

In [227]:
# Doses aplicadas ~ Ciro '18

# Create the OLS model with a different name
model1 = sm.OLS(y1, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     426.0
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:40   Log-Likelihood:                -1016.7
No. Observations:                5507   AIC:                             2063.
Df Residuals:                    5492   BIC:                             2163.
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.3692      0.447      5.306      0.000       1.494       3.244
Ciro Share 18            -1.2296      0.087    -14.115      0.000      -1.400      -1.059
elderly(>70yo)            0.0549      0.003     16.576      0.000       0.048       0.061
Rural pop.               -0.0010      0.000     -3.535      0.000      -0.002      -0.000
White                    -0.0182      0.004     -4.119      0.000      -0.027      -0.010
Black                    -0.0214      0.005     -4.559      0.000      -0.031      -0.012
Brown                    -0.0234      0.004     -5.246      0.000      -0.032      -0.015
Indigenous               -0.0224      0.004     -4.996      0.000      -0.031      -0.014
Pop. density           1.527e-05   1.23e-05      1.237      0.216   -8.93e-06    3.95e-05
Catholic                  0.9971      0.065     15.394      0.000       0.870       1.124
Evangelic                 0.5024      0.104      4.844      0.000       0.299       0.706
Pentecostal Evangelic     0.8892      0.082     10.894      0.000       0.729       1.049
No religion               0.9674      0.134      7.213      0.000       0.705       1.230
Education 11-14y          0.0108      0.002      6.456      0.000       0.008       0.014
Income (BRL)           7.825e-05   1.59e-05      4.924      0.000    4.71e-05       0.000
==============================================================================
Omnibus:                     4575.596   Durbin-Watson:                   1.648
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           836022.273
Skew:                           3.191   Prob(JB):                         0.00
Kurtosis:                      63.023   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [228]:
# Doses recebidas ~ Ciro '18

# Create the OLS model with a different name
model2 = sm.OLS(y2, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     392.3
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:40   Log-Likelihood:                -371.60
No. Observations:                5507   AIC:                             773.2
Df Residuals:                    5492   BIC:                             872.4
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.1713      0.425      5.113      0.000       1.339       3.003
Ciro Share 18             0.0043      0.062      0.069      0.945      -0.118       0.127
elderly(>70yo)            0.0602      0.004     13.638      0.000       0.052       0.069
Rural pop.               -0.0013      0.000     -5.073      0.000      -0.002      -0.001
White                    -0.0191      0.004     -4.734      0.000      -0.027      -0.011
Black                    -0.0239      0.004     -5.561      0.000      -0.032      -0.015
Brown                    -0.0238      0.004     -5.839      0.000      -0.032      -0.016
Indigenous               -0.0227      0.004     -5.317      0.000      -0.031      -0.014
Pop. density          -5.678e-06   7.65e-06     -0.742      0.458   -2.07e-05    9.31e-06
Catholic                  1.2104      0.122      9.946      0.000       0.972       1.449
Evangelic                 0.7247      0.148      4.909      0.000       0.435       1.014
Pentecostal Evangelic     1.1298      0.184      6.130      0.000       0.769       1.491
No religion               1.0306      0.169      6.109      0.000       0.700       1.361
Education 11-14y          0.0089      0.002      5.272      0.000       0.006       0.012
Income (BRL)           9.465e-05    1.6e-05      5.923      0.000    6.33e-05       0.000
==============================================================================
Omnibus:                      758.640   Durbin-Watson:                   1.843
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3718.060
Skew:                           0.576   Prob(JB):                         0.00
Kurtosis:                       6.857   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [229]:
# 3ª dose aplicada ~ Ciro '18

# Create the OLS model with a different name
model3 = sm.OLS(y3, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.444
Model:                            OLS   Adj. R-squared:                  0.443
Method:                 Least Squares   F-statistic:                     303.6
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:40   Log-Likelihood:                 7501.5
No. Observations:                5507   AIC:                        -1.497e+04
Df Residuals:                    5492   BIC:                        -1.487e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0016      0.099     -0.016      0.987      -0.195       0.192
Ciro Share 18            -0.0495      0.013     -3.726      0.000      -0.076      -0.023
elderly(>70yo)            0.0174      0.001     21.943      0.000       0.016       0.019
Rural pop.                0.0002   6.27e-05      2.875      0.004    5.73e-05       0.000
White                    -0.0003      0.001     -0.277      0.782      -0.002       0.002
Black                    -0.0016      0.001     -1.509      0.131      -0.004       0.000
Brown                    -0.0012      0.001     -1.267      0.205      -0.003       0.001
Indigenous               -0.0004      0.001     -0.419      0.675      -0.002       0.002
Pop. density          -7.904e-07    1.3e-06     -0.606      0.544   -3.35e-06    1.77e-06
Catholic                  0.0884      0.014      6.364      0.000       0.061       0.116
Evangelic                 0.0118      0.027      0.437      0.662      -0.041       0.065
Pentecostal Evangelic     0.1274      0.027      4.650      0.000       0.074       0.181
No religion               0.0910      0.026      3.470      0.001       0.040       0.142
Education 11-14y          0.0019      0.000      4.439      0.000       0.001       0.003
Income (BRL)           1.771e-05   4.07e-06      4.353      0.000    9.74e-06    2.57e-05
==============================================================================
Omnibus:                     2129.740   Durbin-Watson:                   1.794
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23326.343
Skew:                           1.534   Prob(JB):                         0.00
Kurtosis:                      12.604   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [230]:
# 3ª dose recebida ~ Ciro '18

# Create the OLS model with a different name
model4 = sm.OLS(y4, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     349.5
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:40   Log-Likelihood:                 7962.1
No. Observations:                5507   AIC:                        -1.589e+04
Df Residuals:                    5492   BIC:                        -1.579e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.0048      0.091      0.053      0.958      -0.173       0.182
Ciro Share 18            -0.0096      0.014     -0.697      0.486      -0.037       0.017
elderly(>70yo)            0.0171      0.001     22.690      0.000       0.016       0.019
Rural pop.                0.0001    5.5e-05      1.938      0.053   -1.21e-06       0.000
White                    -0.0004      0.001     -0.493      0.622      -0.002       0.001
Black                    -0.0018      0.001     -1.866      0.062      -0.004     8.8e-05
Brown                    -0.0014      0.001     -1.607      0.108      -0.003       0.000
Indigenous               -0.0005      0.001     -0.583      0.560      -0.002       0.001
Pop. density           -1.16e-06   1.26e-06     -0.917      0.359   -3.64e-06    1.32e-06
Catholic                  0.1022      0.015      6.608      0.000       0.072       0.133
Evangelic                 0.0269      0.026      1.015      0.310      -0.025       0.079
Pentecostal Evangelic     0.1381      0.030      4.604      0.000       0.079       0.197
No religion               0.0959      0.025      3.786      0.000       0.046       0.146
Education 11-14y          0.0019      0.000      5.173      0.000       0.001       0.003
Income (BRL)           1.814e-05   3.65e-06      4.962      0.000     1.1e-05    2.53e-05
==============================================================================
Omnibus:                     1553.478   Durbin-Watson:                   1.795
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10114.764
Skew:                           1.186   Prob(JB):                         0.00
Kurtosis:                       9.202   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [231]:
# Cobertura vacinal ~ Ciro '18

# Create the OLS model with a different name
model5 = sm.OLS(y5, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.175
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     73.33
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          3.98e-192
Time:                        13:23:40   Log-Likelihood:                -24829.
No. Observations:                5507   AIC:                         4.969e+04
Df Residuals:                    5492   BIC:                         4.979e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   174.9543     33.438      5.232      0.000     109.418     240.491
Ciro Share 18           -11.1514      4.170     -2.674      0.007     -19.324      -2.979
elderly(>70yo)            2.0661      0.263      7.854      0.000       1.551       2.582
Rural pop.               -0.0120      0.021     -0.565      0.572      -0.054       0.030
White                    -1.0830      0.334     -3.239      0.001      -1.738      -0.428
Black                    -1.3120      0.353     -3.715      0.000      -2.004      -0.620
Brown                    -1.2756      0.337     -3.785      0.000      -1.936      -0.615
Indigenous               -1.4406      0.339     -4.245      0.000      -2.106      -0.775
Pop. density             -0.0022      0.001     -3.388      0.001      -0.003      -0.001
Catholic                 10.2713      3.141      3.271      0.001       4.116      16.427
Evangelic                 4.6085      7.950      0.580      0.562     -10.974      20.191
Pentecostal Evangelic    20.5847      6.269      3.284      0.001       8.298      32.871
No religion             -60.3480      9.680     -6.234      0.000     -79.320     -41.375
Education 11-14y         -0.9245      0.135     -6.872      0.000      -1.188      -0.661
Income (BRL)              0.0085      0.001      6.287      0.000       0.006       0.011
==============================================================================
Omnibus:                      548.771   Durbin-Watson:                   1.862
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2018.498
Skew:                           0.463   Prob(JB):                         0.00
Kurtosis:                       5.818   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [232]:
# Cobertura contra Polio ~ Ciro '18

# Create the OLS model with a different name
model6 = sm.OLS(y6, X4).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     43.10
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          2.74e-113
Time:                        13:23:40   Log-Likelihood:                -26198.
No. Observations:                5507   AIC:                         5.243e+04
Df Residuals:                    5492   BIC:                         5.252e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   196.9032     43.142      4.564      0.000     112.346     281.460
Ciro Share 18            -5.9853      4.990     -1.200      0.230     -15.765       3.794
elderly(>70yo)            2.5189      0.352      7.151      0.000       1.829       3.209
Rural pop.               -0.0052      0.026     -0.199      0.842      -0.057       0.046
White                    -1.1759      0.433     -2.715      0.007      -2.025      -0.327
Black                    -1.4463      0.456     -3.172      0.002      -2.340      -0.553
Brown                    -1.3820      0.435     -3.174      0.002      -2.236      -0.529
Indigenous               -1.6649      0.436     -3.819      0.000      -2.519      -0.810
Pop. density             -0.0018      0.001     -2.453      0.014      -0.003      -0.000
Catholic                 11.2663      3.991      2.823      0.005       3.445      19.088
Evangelic                -4.6592     12.950     -0.360      0.719     -30.041      20.723
Pentecostal Evangelic    16.5370      8.263      2.001      0.045       0.341      32.733
No religion             -47.7725     12.443     -3.839      0.000     -72.160     -23.385
Education 11-14y         -1.0096      0.172     -5.879      0.000      -1.346      -0.673
Income (BRL)              0.0061      0.002      3.453      0.001       0.003       0.010
==============================================================================
Omnibus:                     2153.719   Durbin-Watson:                   1.926
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33378.868
Skew:                           1.448   Prob(JB):                         0.00
Kurtosis:                      14.708   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [233]:
table5 = Stargazer([model1, model2, model3, model4, model5, model6])
table5.covariate_order(['Ciro Share 18', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table5.show_degrees_of_freedom(False)
print(table5.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Ciro Share 18 & -1.230$^{***}$ & 0.004$^{}$ & -0.050$^{***}$ & -0.010$^{}$ & -11.151$^{***}$ & -5.985$^{}$ \\
  & (0.087) & (0.062) & (0.013) & (0.014) & (4.170) & (4.990) \\
 elderly(>70yo) & 0.055$^{***}$ & 0.060$^{***}$ & 0.017$^{***}$ & 0.017$^{***}$ & 2.066$^{***}$ & 2.519$^{***}$ \\
  & (0.003) & (0.004) & (0.001) & (0.001) & (0.263) & (0.352) \\
 Rural pop. & -0.001$^{***}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{*}$ & -0.012$^{}$ & -0.005$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & 0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.002$^{**}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.018$^{***}$ & -0.019$^{***}$ & -0.000$^{}

In [234]:
# Doses aplicadas ~ Tebet '22

# Create the OLS model with a different name
model1 = sm.OLS(y1, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.459
Method:                 Least Squares   F-statistic:                     414.3
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:40   Log-Likelihood:                -1266.8
No. Observations:                5507   AIC:                             2564.
Df Residuals:                    5492   BIC:                             2663.
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.0574      0.458      4.494      0.000       1.160       2.955
Tebet Share 22            3.3268      0.333      9.980      0.000       2.673       3.980
elderly(>70yo)            0.0450      0.003     13.220      0.000       0.038       0.052
Rural pop.               -0.0003      0.000     -0.986      0.324      -0.001       0.000
White                    -0.0182      0.004     -4.055      0.000      -0.027      -0.009
Black                    -0.0184      0.005     -3.846      0.000      -0.028      -0.009
Brown                    -0.0231      0.005     -5.090      0.000      -0.032      -0.014
Indigenous               -0.0210      0.005     -4.584      0.000      -0.030      -0.012
Pop. density          -1.477e-05   1.08e-05     -1.367      0.171   -3.59e-05     6.4e-06
Catholic                  1.0334      0.074     13.937      0.000       0.888       1.179
Evangelic                 0.6992      0.108      6.459      0.000       0.487       0.911
Pentecostal Evangelic     1.1192      0.108     10.346      0.000       0.907       1.331
No religion               1.0928      0.141      7.766      0.000       0.817       1.369
Education 11-14y          0.0084      0.002      4.834      0.000       0.005       0.012
Income (BRL)           9.808e-05   1.51e-05      6.513      0.000    6.86e-05       0.000
==============================================================================
Omnibus:                     3489.897   Durbin-Watson:                   1.434
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           540704.390
Skew:                           2.035   Prob(JB):                         0.00
Kurtosis:                      51.372   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [235]:
# Doses recebidas ~ Tebet '22

# Create the OLS model with a different name
model2 = sm.OLS(y2, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.566
Model:                            OLS   Adj. R-squared:                  0.565
Method:                 Least Squares   F-statistic:                     394.6
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:40   Log-Likelihood:                -351.49
No. Observations:                5507   AIC:                             733.0
Df Residuals:                    5492   BIC:                             832.2
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.1841      0.422      5.181      0.000       1.358       3.010
Tebet Share 22            1.7288      0.284      6.090      0.000       1.172       2.285
elderly(>70yo)            0.0609      0.004     14.472      0.000       0.053       0.069
Rural pop.               -0.0015      0.000     -5.414      0.000      -0.002      -0.001
White                    -0.0196      0.004     -4.887      0.000      -0.027      -0.012
Black                    -0.0244      0.004     -5.703      0.000      -0.033      -0.016
Brown                    -0.0239      0.004     -5.874      0.000      -0.032      -0.016
Indigenous               -0.0228      0.004     -5.358      0.000      -0.031      -0.014
Pop. density          -6.909e-06   7.25e-06     -0.953      0.341   -2.11e-05     7.3e-06
Catholic                  1.2158      0.122      9.953      0.000       0.976       1.455
Evangelic                 0.7269      0.145      5.023      0.000       0.443       1.011
Pentecostal Evangelic     1.0744      0.177      6.081      0.000       0.728       1.421
No religion               1.0383      0.168      6.174      0.000       0.709       1.368
Education 11-14y          0.0078      0.002      4.582      0.000       0.004       0.011
Income (BRL)           7.705e-05   1.53e-05      5.038      0.000    4.71e-05       0.000
==============================================================================
Omnibus:                      771.411   Durbin-Watson:                   1.854
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3880.937
Skew:                           0.579   Prob(JB):                         0.00
Kurtosis:                       6.946   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [236]:
# 3ª dose aplicada ~ Tebet '22

# Create the OLS model with a different name
model3 = sm.OLS(y3, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.448
Model:                            OLS   Adj. R-squared:                  0.447
Method:                 Least Squares   F-statistic:                     304.1
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:41   Log-Likelihood:                 7522.1
No. Observations:                5507   AIC:                        -1.501e+04
Df Residuals:                    5492   BIC:                        -1.492e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0115      0.099     -0.117      0.907      -0.205       0.182
Tebet Share 22            0.5231      0.075      6.948      0.000       0.376       0.671
elderly(>70yo)            0.0171      0.001     22.027      0.000       0.016       0.019
Rural pop.                0.0002   6.18e-05      2.907      0.004    5.86e-05       0.000
White                    -0.0004      0.001     -0.411      0.681      -0.002       0.001
Black                    -0.0015      0.001     -1.503      0.133      -0.004       0.000
Brown                    -0.0013      0.001     -1.282      0.200      -0.003       0.001
Indigenous               -0.0004      0.001     -0.390      0.697      -0.002       0.002
Pop. density          -2.299e-06   1.19e-06     -1.939      0.053   -4.62e-06    2.49e-08
Catholic                  0.0911      0.015      6.084      0.000       0.062       0.121
Evangelic                 0.0204      0.027      0.744      0.457      -0.033       0.074
Pentecostal Evangelic     0.1245      0.028      4.370      0.000       0.069       0.180
No religion               0.0978      0.027      3.677      0.000       0.046       0.150
Education 11-14y          0.0015      0.000      3.563      0.000       0.001       0.002
Income (BRL)           1.459e-05   4.09e-06      3.564      0.000    6.57e-06    2.26e-05
==============================================================================
Omnibus:                     2130.561   Durbin-Watson:                   1.801
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            24482.526
Skew:                           1.520   Prob(JB):                         0.00
Kurtosis:                      12.872   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [237]:
# 3ª dose recebida ~ Tebet '22

# Create the OLS model with a different name
model4 = sm.OLS(y4, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     349.7
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:41   Log-Likelihood:                 7994.2
No. Observations:                5507   AIC:                        -1.596e+04
Df Residuals:                    5492   BIC:                        -1.586e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.0054      0.090      0.060      0.952      -0.172       0.182
Tebet Share 22            0.4840      0.068      7.096      0.000       0.350       0.618
elderly(>70yo)            0.0172      0.001     23.328      0.000       0.016       0.019
Rural pop.             7.782e-05    5.5e-05      1.414      0.157   -3.01e-05       0.000
White                    -0.0006      0.001     -0.663      0.507      -0.002       0.001
Black                    -0.0019      0.001     -1.986      0.047      -0.004   -2.43e-05
Brown                    -0.0015      0.001     -1.637      0.102      -0.003       0.000
Indigenous               -0.0005      0.001     -0.606      0.544      -0.002       0.001
Pop. density          -1.746e-06   1.17e-06     -1.486      0.137   -4.05e-06    5.57e-07
Catholic                  0.1040      0.016      6.535      0.000       0.073       0.135
Evangelic                 0.0292      0.027      1.101      0.271      -0.023       0.081
Pentecostal Evangelic     0.1255      0.029      4.257      0.000       0.068       0.183
No religion               0.0990      0.025      3.886      0.000       0.049       0.149
Education 11-14y          0.0016      0.000      4.308      0.000       0.001       0.002
Income (BRL)           1.368e-05   3.65e-06      3.742      0.000    6.51e-06    2.08e-05
==============================================================================
Omnibus:                     1575.043   Durbin-Watson:                   1.815
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10822.256
Skew:                           1.188   Prob(JB):                         0.00
Kurtosis:                       9.444   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [238]:
# Cobertura vacinal ~ Tebet '22

# Create the OLS model with a different name
model5 = sm.OLS(y5, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.172
Method:                 Least Squares   F-statistic:                     73.44
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          2.13e-192
Time:                        13:23:41   Log-Likelihood:                -24833.
No. Observations:                5507   AIC:                         4.970e+04
Df Residuals:                    5492   BIC:                         4.980e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   171.7931     33.334      5.154      0.000     106.460     237.126
Tebet Share 22          -19.1085     23.489     -0.814      0.416     -65.146      26.929
elderly(>70yo)            1.9559      0.260      7.526      0.000       1.446       2.465
Rural pop.               -0.0018      0.021     -0.086      0.932      -0.043       0.039
White                    -1.0672      0.334     -3.197      0.001      -1.722      -0.413
Black                    -1.2711      0.352     -3.609      0.000      -1.961      -0.581
Brown                    -1.2699      0.336     -3.776      0.000      -1.929      -0.611
Indigenous               -1.4248      0.339     -4.208      0.000      -2.088      -0.761
Pop. density             -0.0024      0.001     -3.730      0.000      -0.004      -0.001
Catholic                 10.4438      3.199      3.265      0.001       4.174      16.714
Evangelic                 6.3111      7.956      0.793      0.428      -9.282      21.904
Pentecostal Evangelic    24.2171      6.293      3.848      0.000      11.882      36.552
No religion             -59.4401      9.705     -6.125      0.000     -78.462     -40.418
Education 11-14y         -0.9143      0.136     -6.708      0.000      -1.181      -0.647
Income (BRL)              0.0091      0.001      6.728      0.000       0.006       0.012
==============================================================================
Omnibus:                      540.434   Durbin-Watson:                   1.860
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1942.351
Skew:                           0.462   Prob(JB):                         0.00
Kurtosis:                       5.759   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [239]:
# Cobertura contra Polio ~ Tebet '22

# Create the OLS model with a different name
model6 = sm.OLS(y6, X5).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.113
Method:                 Least Squares   F-statistic:                     43.12
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          2.42e-113
Time:                        13:23:41   Log-Likelihood:                -26198.
No. Observations:                5507   AIC:                         5.243e+04
Df Residuals:                    5492   BIC:                         5.253e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   195.1379     43.020      4.536      0.000     110.821     279.455
Tebet Share 22          -20.4040     30.455     -0.670      0.503     -80.095      39.287
elderly(>70yo)            2.4555      0.345      7.113      0.000       1.779       3.132
Rural pop.                0.0010      0.026      0.039      0.969      -0.050       0.052
White                    -1.1641      0.433     -2.691      0.007      -2.012      -0.316
Black                    -1.4215      0.455     -3.126      0.002      -2.313      -0.530
Brown                    -1.3784      0.435     -3.171      0.002      -2.230      -0.526
Indigenous               -1.6557      0.435     -3.805      0.000      -2.509      -0.803
Pop. density             -0.0019      0.001     -2.635      0.008      -0.003      -0.000
Catholic                 11.3266      4.013      2.823      0.005       3.462      19.192
Evangelic                -3.7623     12.935     -0.291      0.771     -29.114      21.590
Pentecostal Evangelic    18.8049      8.243      2.281      0.023       2.649      34.961
No religion             -47.3326     12.438     -3.805      0.000     -71.711     -22.954
Education 11-14y         -0.9975      0.174     -5.740      0.000      -1.338      -0.657
Income (BRL)              0.0066      0.002      3.640      0.000       0.003       0.010
==============================================================================
Omnibus:                     2153.894   Durbin-Watson:                   1.925
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33327.584
Skew:                           1.449   Prob(JB):                         0.00
Kurtosis:                      14.698   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [240]:
table6 = Stargazer([model1, model2, model3, model4, model5, model6])
table6.covariate_order(['Tebet Share 22', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table6.show_degrees_of_freedom(False)
print(table6.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Tebet Share 22 & 3.327$^{***}$ & 1.729$^{***}$ & 0.523$^{***}$ & 0.484$^{***}$ & -19.109$^{}$ & -20.404$^{}$ \\
  & (0.333) & (0.284) & (0.075) & (0.068) & (23.489) & (30.455) \\
 elderly(>70yo) & 0.045$^{***}$ & 0.061$^{***}$ & 0.017$^{***}$ & 0.017$^{***}$ & 1.956$^{***}$ & 2.456$^{***}$ \\
  & (0.003) & (0.004) & (0.001) & (0.001) & (0.260) & (0.345) \\
 Rural pop. & -0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{}$ & -0.002$^{}$ & 0.001$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{*}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.002$^{***}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.018$^{***}$ & -0.020$^{***}$ & -0.000$^

In [241]:
# Doses aplicadas ~ Bolsonaro '18

# Create the OLS model with a different name
model1 = sm.OLS(y1, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model1)

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.452
Method:                 Least Squares   F-statistic:                     413.6
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:41   Log-Likelihood:                -1300.0
No. Observations:                5507   AIC:                             2630.
Df Residuals:                    5492   BIC:                             2729.
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.8861      0.459      4.107      0.000       0.986       2.786
Bolsonaro Share 18        0.2539      0.046      5.493      0.000       0.163       0.345
elderly(>70yo)            0.0442      0.003     13.039      0.000       0.038       0.051
Rural pop.             8.816e-05      0.000      0.311      0.756      -0.000       0.001
White                    -0.0169      0.005     -3.761      0.000      -0.026      -0.008
Black                    -0.0162      0.005     -3.361      0.001      -0.026      -0.007
Brown                    -0.0217      0.005     -4.767      0.000      -0.031      -0.013
Indigenous               -0.0197      0.005     -4.293      0.000      -0.029      -0.011
Pop. density          -4.847e-06    1.1e-05     -0.439      0.661   -2.65e-05    1.68e-05
Catholic                  1.0671      0.072     14.896      0.000       0.927       1.207
Evangelic                 0.6655      0.104      6.421      0.000       0.462       0.869
Pentecostal Evangelic     1.0485      0.103     10.132      0.000       0.846       1.251
No religion               1.1080      0.137      8.085      0.000       0.839       1.377
Education 11-14y          0.0099      0.002      5.715      0.000       0.006       0.013
Income (BRL)           9.087e-05   1.58e-05      5.769      0.000       6e-05       0.000
==============================================================================
Omnibus:                     3365.538   Durbin-Watson:                   1.417
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           510094.336
Skew:                           1.918   Prob(JB):                         0.00
Kurtosis:                      49.993   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [242]:
# Doses recebidas ~ Bolsonaro '18

# Create the OLS model with a different name
model2 = sm.OLS(y2, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model2)

model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     393.3
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:41   Log-Likelihood:                -364.42
No. Observations:                5507   AIC:                             758.8
Df Residuals:                    5492   BIC:                             858.0
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     2.0970      0.419      5.008      0.000       1.276       2.918
Bolsonaro Share 18        0.1288      0.038      3.392      0.001       0.054       0.203
elderly(>70yo)            0.0605      0.004     14.534      0.000       0.052       0.069
Rural pop.               -0.0013      0.000     -4.754      0.000      -0.002      -0.001
White                    -0.0190      0.004     -4.733      0.000      -0.027      -0.011
Black                    -0.0233      0.004     -5.455      0.000      -0.032      -0.015
Brown                    -0.0232      0.004     -5.719      0.000      -0.031      -0.015
Indigenous               -0.0221      0.004     -5.232      0.000      -0.030      -0.014
Pop. density          -1.846e-06   7.49e-06     -0.246      0.805   -1.65e-05    1.28e-05
Catholic                  1.2328      0.119     10.377      0.000       1.000       1.466
Evangelic                 0.7097      0.142      5.009      0.000       0.432       0.987
Pentecostal Evangelic     1.0398      0.181      5.743      0.000       0.685       1.395
No religion               1.0458      0.166      6.316      0.000       0.721       1.370
Education 11-14y          0.0086      0.002      5.092      0.000       0.005       0.012
Income (BRL)           7.381e-05   1.65e-05      4.472      0.000    4.15e-05       0.000
==============================================================================
Omnibus:                      736.683   Durbin-Watson:                   1.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3440.236
Skew:                           0.570   Prob(JB):                         0.00
Kurtosis:                       6.700   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [243]:
# 3ª dose aplicada ~ Bolsonaro '18

# Create the OLS model with a different name
model3 = sm.OLS(y3, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model3)

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.442
Model:                            OLS   Adj. R-squared:                  0.441
Method:                 Least Squares   F-statistic:                     302.4
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:41   Log-Likelihood:                 7490.9
No. Observations:                5507   AIC:                        -1.495e+04
Df Residuals:                    5492   BIC:                        -1.485e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0214      0.098     -0.219      0.827      -0.213       0.170
Bolsonaro Share 18        0.0109      0.009      1.169      0.242      -0.007       0.029
elderly(>70yo)            0.0169      0.001     21.825      0.000       0.015       0.018
Rural pop.                0.0002   6.17e-05      3.636      0.000       0.000       0.000
White                    -0.0002      0.001     -0.226      0.821      -0.002       0.002
Black                    -0.0013      0.001     -1.311      0.190      -0.003       0.001
Brown                    -0.0012      0.001     -1.202      0.230      -0.003       0.001
Indigenous               -0.0003      0.001     -0.307      0.759      -0.002       0.002
Pop. density          -1.582e-06   1.29e-06     -1.224      0.221   -4.12e-06    9.51e-07
Catholic                  0.0914      0.014      6.421      0.000       0.063       0.119
Evangelic                 0.0183      0.027      0.678      0.498      -0.035       0.071
Pentecostal Evangelic     0.1334      0.029      4.522      0.000       0.076       0.191
No religion               0.0967      0.026      3.701      0.000       0.045       0.148
Education 11-14y          0.0018      0.000      4.313      0.000       0.001       0.003
Income (BRL)           1.811e-05   4.41e-06      4.111      0.000    9.48e-06    2.67e-05
==============================================================================
Omnibus:                     2077.034   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22370.186
Skew:                           1.492   Prob(JB):                         0.00
Kurtosis:                      12.412   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [244]:
# 3ª dose recebida ~ Bolsonaro '18

# Create the OLS model with a different name
model4 = sm.OLS(y4, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model4)

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.497
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     348.0
Date:                Tue, 04 Apr 2023   Prob (F-statistic):               0.00
Time:                        13:23:41   Log-Likelihood:                 7963.3
No. Observations:                5507   AIC:                        -1.590e+04
Df Residuals:                    5492   BIC:                        -1.580e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    -0.0059      0.090     -0.065      0.948      -0.182       0.170
Bolsonaro Share 18        0.0137      0.008      1.757      0.079      -0.002       0.029
elderly(>70yo)            0.0170      0.001     23.213      0.000       0.016       0.018
Rural pop.                0.0001   5.53e-05      2.180      0.029    1.22e-05       0.000
White                    -0.0004      0.001     -0.474      0.635      -0.002       0.001
Black                    -0.0017      0.001     -1.772      0.076      -0.003       0.000
Brown                    -0.0014      0.001     -1.536      0.125      -0.003       0.000
Indigenous               -0.0005      0.001     -0.506      0.613      -0.002       0.001
Pop. density          -9.758e-07   1.25e-06     -0.781      0.435   -3.42e-06    1.47e-06
Catholic                  0.1048      0.015      6.958      0.000       0.075       0.134
Evangelic                 0.0268      0.026      1.032      0.302      -0.024       0.078
Pentecostal Evangelic     0.1313      0.030      4.306      0.000       0.072       0.191
No religion               0.0984      0.025      3.943      0.000       0.050       0.147
Education 11-14y          0.0019      0.000      5.076      0.000       0.001       0.003
Income (BRL)           1.635e-05   3.89e-06      4.207      0.000    8.73e-06     2.4e-05
==============================================================================
Omnibus:                     1545.370   Durbin-Watson:                   1.796
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            10046.981
Skew:                           1.179   Prob(JB):                         0.00
Kurtosis:                       9.183   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [245]:
# Cobertura vacinal ~ Bolsonaro '18

# Create the OLS model with a different name
model5 = sm.OLS(y5, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model5)

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.179
Method:                 Least Squares   F-statistic:                     74.38
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          8.79e-195
Time:                        13:23:41   Log-Likelihood:                -24808.
No. Observations:                5507   AIC:                         4.965e+04
Df Residuals:                    5492   BIC:                         4.975e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   159.8916     33.572      4.763      0.000      94.091     225.692
Bolsonaro Share 18       20.5304      3.114      6.592      0.000      14.427      26.634
elderly(>70yo)            2.0126      0.258      7.811      0.000       1.508       2.518
Rural pop.                0.0064      0.021      0.305      0.761      -0.035       0.047
White                    -1.0591      0.336     -3.154      0.002      -1.717      -0.401
Black                    -1.1731      0.354     -3.310      0.001      -1.868      -0.478
Brown                    -1.1762      0.339     -3.473      0.001      -1.840      -0.512
Indigenous               -1.3370      0.341     -3.921      0.000      -2.005      -0.669
Pop. density             -0.0018      0.001     -2.859      0.004      -0.003      -0.001
Catholic                 14.0830      3.259      4.321      0.000       7.695      20.471
Evangelic                 4.0686      7.908      0.515      0.607     -11.430      19.568
Pentecostal Evangelic     9.4593      6.386      1.481      0.139      -3.058      21.976
No religion             -56.8700      9.728     -5.846      0.000     -75.936     -37.804
Education 11-14y         -0.9843      0.135     -7.311      0.000      -1.248      -0.720
Income (BRL)              0.0057      0.001      4.047      0.000       0.003       0.008
==============================================================================
Omnibus:                      568.792   Durbin-Watson:                   1.882
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2215.728
Skew:                           0.465   Prob(JB):                         0.00
Kurtosis:                       5.965   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [246]:
# Cobertura contra Polio ~ Bolsonaro '18

# Create the OLS model with a different name
model6 = sm.OLS(y6, X6).fit(cov_type = "HC0")

# Add the model name to the list of models
models.append(model6)

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                     44.31
Date:                Tue, 04 Apr 2023   Prob (F-statistic):          1.56e-116
Time:                        13:23:41   Log-Likelihood:                -26184.
No. Observations:                5507   AIC:                         5.240e+04
Df Residuals:                    5492   BIC:                         5.250e+04
Df Model:                          14                                         
Covariance Type:                  HC0                                         
=========================================================================================
                            coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                   183.7717     43.328      4.241      0.000      98.850     268.694
Bolsonaro Share 18       19.6319      4.134      4.749      0.000      11.530      27.734
elderly(>70yo)            2.5107      0.346      7.266      0.000       1.833       3.188
Rural pop.                0.0087      0.026      0.334      0.738      -0.042       0.060
White                    -1.1570      0.435     -2.659      0.008      -2.010      -0.304
Black                    -1.3284      0.457     -2.905      0.004      -2.225      -0.432
Brown                    -1.2889      0.438     -2.945      0.003      -2.147      -0.431
Indigenous               -1.5719      0.438     -3.588      0.000      -2.431      -0.713
Pop. density             -0.0014      0.001     -1.836      0.066      -0.003    9.26e-05
Catholic                 14.8133      4.095      3.618      0.000       6.788      22.838
Evangelic                -5.9030     12.951     -0.456      0.649     -31.287      19.481
Pentecostal Evangelic     4.6262      8.412      0.550      0.582     -11.861      21.113
No religion             -44.8650     12.515     -3.585      0.000     -69.393     -20.337
Education 11-14y         -1.0658      0.173     -6.168      0.000      -1.405      -0.727
Income (BRL)              0.0033      0.002      1.776      0.076      -0.000       0.007
==============================================================================
Omnibus:                     2177.904   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34729.974
Skew:                           1.461   Prob(JB):                         0.00
Kurtosis:                      14.951   Cond. No.                     1.67e+05
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.67e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [247]:
table7 = Stargazer([model1, model2, model3, model4, model5, model6])
table7.covariate_order(['Bolsonaro Share 18', 'elderly(>70yo)', 'Rural pop.', 'Pop. density', 'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
           'Pentecostal Evangelic', 'No religion', 'Education 11-14y', 'Income (BRL)'])
table7.show_degrees_of_freedom(False)
print(table7.render_latex())

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{6}{c}{\textit{Dependent variable:}} \
\cr \cline{6-7}
\\[-1.8ex] & (1) & (2) & (3) & (4) & (5) & (6) \\
\hline \\[-1.8ex]
 Bolsonaro Share 18 & 0.254$^{***}$ & 0.129$^{***}$ & 0.011$^{}$ & 0.014$^{*}$ & 20.530$^{***}$ & 19.632$^{***}$ \\
  & (0.046) & (0.038) & (0.009) & (0.008) & (3.114) & (4.134) \\
 elderly(>70yo) & 0.044$^{***}$ & 0.061$^{***}$ & 0.017$^{***}$ & 0.017$^{***}$ & 2.013$^{***}$ & 2.511$^{***}$ \\
  & (0.003) & (0.004) & (0.001) & (0.001) & (0.258) & (0.346) \\
 Rural pop. & 0.000$^{}$ & -0.001$^{***}$ & 0.000$^{***}$ & 0.000$^{**}$ & 0.006$^{}$ & 0.009$^{}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.021) & (0.026) \\
 Pop. density & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.000$^{}$ & -0.002$^{***}$ & -0.001$^{*}$ \\
  & (0.000) & (0.000) & (0.000) & (0.000) & (0.001) & (0.001) \\
 White & -0.017$^{***}$ & -0.019$^{***}$ & -0.000$^{}

------------------------
Descriptive statistics

In [248]:
df.columns

Index(['id_municipio', 'id_municipio_tse', 'Name', 'State', 'Region',
       'Population', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose4 Applied', 'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Dose4 Received', 'Doses Applied', 'Doses Received',
       'Bolsonaro Share 18', 'Bolsonaro Share 22', 'Far-left 18',
       'Far-right18', 'PSL coligation 18', 'Ciro Share 18', 'Far-left 22',
       'Far-right 22', 'PL coligation 22', 'Tebet Share 22',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage',
       'elderly(>70yo)', 'Rural pop.', 'White', 'Black', 'Brown', 'Indigenous',
       'Pop. density', 'Catholic', 'Evangelic', 'Pentecostal Evangelic',
       'Umbanda/Candomble', 'No religion', 'Education 8-10y',
       'Education 11-14y', 'Education 15y plus', 'Income (BRL)',
       'Education 8y less', 'Outer wing 18', 'outer wing 22'],
      dtype='object')

In [249]:
print(df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &        max \\
\midrule
Bolsonaro Share 22    &  5507.0 &     0.439 &    0.187 &    0.061 &      0.890 \\
Far-left 18           &  5507.0 &     0.004 &    0.002 &    0.000 &      0.025 \\
Far-right 22          &  5507.0 &     0.004 &    0.004 &    0.000 &      0.032 \\
Dose1 Applied         &  5507.0 &     0.754 &    0.176 &    0.009 &      3.998 \\
Dose2 Applied         &  5507.0 &     0.620 &    0.182 &    0.000 &      3.257 \\
Dose3 Applied         &  5507.0 &     0.139 &    0.083 &    0.000 &      0.827 \\
Dose1 Received        &  5507.0 &     0.793 &    0.159 &    0.179 &      3.120 \\
Dose2 Received        &  5507.0 &     0.648 &    0.172 &    0.085 &      2.811 \\
Dose3 Received        &  5507.0 &     0.140 &    0.080 &    0.004 &      0.744 \\
Vaccination coverage  &  5507.0 &    76.939 &   24.189 &    2.280 &    245.580 \\
BCG coverage          &  5507.0 &    73.169 &   36.321 &    0.000 &    772.

<ipython-input-249-c504c304be65>:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [250]:
df['Region'].unique()

array(['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste'],
      dtype=object)

In [251]:
filtered_df = df[df['Region'] == 'Norte']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  449.0 &     0.454 &    0.166 &    0.086 &     0.829 \\
Far-left 18           &  449.0 &     0.005 &    0.003 &    0.000 &     0.020 \\
Far-right 22          &  449.0 &     0.001 &    0.001 &    0.000 &     0.006 \\
Dose1 Applied         &  449.0 &     0.617 &    0.164 &    0.175 &     2.252 \\
Dose2 Applied         &  449.0 &     0.446 &    0.145 &    0.051 &     1.197 \\
Dose3 Applied         &  449.0 &     0.068 &    0.047 &    0.001 &     0.406 \\
Dose1 Received        &  449.0 &     0.617 &    0.173 &    0.179 &     1.636 \\
Dose2 Received        &  449.0 &     0.445 &    0.151 &    0.085 &     0.961 \\
Dose3 Received        &  449.0 &     0.066 &    0.043 &    0.004 &     0.309 \\
Vaccination coverage  &  449.0 &    68.026 &   22.440 &   19.030 &   136.230 \\
BCG coverage          &  449.0 &    75.867 &   31.241 &    5.130 &   235.470 \\
Polio coverage   

<ipython-input-251-ff39d407ecb8>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [252]:
filtered_df = df[df['Region'] == 'Nordeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &     mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  1787.0 &    0.235 &    0.089 &    0.061 &     0.623 \\
Far-left 18           &  1787.0 &    0.003 &    0.002 &    0.000 &     0.022 \\
Far-right 22          &  1787.0 &    0.002 &    0.001 &    0.000 &     0.008 \\
Dose1 Applied         &  1787.0 &    0.676 &    0.227 &    0.009 &     3.998 \\
Dose2 Applied         &  1787.0 &    0.530 &    0.205 &    0.000 &     3.257 \\
Dose3 Applied         &  1787.0 &    0.101 &    0.066 &    0.000 &     0.827 \\
Dose1 Received        &  1787.0 &    0.750 &    0.148 &    0.269 &     3.120 \\
Dose2 Received        &  1787.0 &    0.581 &    0.157 &    0.113 &     2.811 \\
Dose3 Received        &  1787.0 &    0.102 &    0.060 &    0.007 &     0.690 \\
Vaccination coverage  &  1787.0 &   68.263 &   21.967 &    5.100 &   214.260 \\
BCG coverage          &  1787.0 &   64.670 &   39.712 &    0.000 &   772.860 \\
Polio coverage   

<ipython-input-252-1965c469d4bc>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [253]:
filtered_df = df[df['Region'] == 'Sudeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &        max \\
\midrule
Bolsonaro Share 22    &  1666.0 &     0.511 &    0.129 &    0.155 &      0.812 \\
Far-left 18           &  1666.0 &     0.004 &    0.003 &    0.000 &      0.025 \\
Far-right 22          &  1666.0 &     0.007 &    0.004 &    0.000 &      0.023 \\
Dose1 Applied         &  1666.0 &     0.806 &    0.093 &    0.420 &      1.609 \\
Dose2 Applied         &  1666.0 &     0.681 &    0.123 &    0.172 &      1.395 \\
Dose3 Applied         &  1666.0 &     0.165 &    0.077 &    0.001 &      0.741 \\
Dose1 Received        &  1666.0 &     0.832 &    0.125 &    0.437 &      1.658 \\
Dose2 Received        &  1666.0 &     0.700 &    0.140 &    0.229 &      1.498 \\
Dose3 Received        &  1666.0 &     0.166 &    0.075 &    0.010 &      0.744 \\
Vaccination coverage  &  1666.0 &    80.422 &   23.858 &    2.280 &    210.880 \\
BCG coverage          &  1666.0 &    73.449 &   32.653 &    0.000 &    230.

<ipython-input-253-77f52f9ad7b6>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [254]:
filtered_df = df[df['Region'] == 'Sul']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &   count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  1159.0 &     0.598 &    0.115 &    0.282 &     0.890 \\
Far-left 18           &  1159.0 &     0.004 &    0.002 &    0.000 &     0.024 \\
Far-right 22          &  1159.0 &     0.005 &    0.003 &    0.000 &     0.032 \\
Dose1 Applied         &  1159.0 &     0.842 &    0.099 &    0.396 &     2.252 \\
Dose2 Applied         &  1159.0 &     0.734 &    0.108 &    0.272 &     1.950 \\
Dose3 Applied         &  1159.0 &     0.188 &    0.083 &    0.006 &     0.503 \\
Dose1 Received        &  1159.0 &     0.864 &    0.129 &    0.377 &     2.575 \\
Dose2 Received        &  1159.0 &     0.751 &    0.131 &    0.296 &     2.177 \\
Dose3 Received        &  1159.0 &     0.189 &    0.082 &    0.022 &     0.509 \\
Vaccination coverage  &  1159.0 &    87.193 &   23.442 &    5.760 &   245.580 \\
BCG coverage          &  1159.0 &    83.753 &   36.360 &    0.000 &   300.000 \\
Polio

<ipython-input-254-f4e8c2d7bdf7>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',


In [255]:
filtered_df = df[df['Region'] == 'Centro-Oeste']

print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
       'Dose1 Received', 'Dose2 Received', 'Dose3 Received',
       'Vaccination coverage', 'BCG coverage', 'Polio coverage', 'elderly(>70yo)', 'Rural pop.', 'Pop. density',
       'White', 'Black', 'Brown', 'Indigenous', 'Catholic', 'Evangelic',
       'Pentecostal Evangelic', 'Umbanda/Candomble', 'No religion',
       'Education 8y less', 'Education 8-10y', 'Education 11-14y', 'Education 15y plus',
       'Income (BRL)']].describe().T.iloc[:, list(range(4)) + [-1]].round(3).to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  count &      mean &      std &      min &       max \\
\midrule
Bolsonaro Share 22    &  446.0 &     0.564 &    0.101 &    0.174 &     0.811 \\
Far-left 18           &  446.0 &     0.004 &    0.002 &    0.000 &     0.014 \\
Far-right 22          &  446.0 &     0.003 &    0.001 &    0.000 &     0.008 \\
Dose1 Applied         &  446.0 &     0.785 &    0.134 &    0.370 &     2.081 \\
Dose2 Applied         &  446.0 &     0.635 &    0.142 &    0.304 &     1.730 \\
Dose3 Applied         &  446.0 &     0.135 &    0.076 &    0.000 &     0.447 \\
Dose1 Received        &  446.0 &     0.809 &    0.184 &    0.378 &     2.484 \\
Dose2 Received        &  446.0 &     0.655 &    0.171 &    0.297 &     1.990 \\
Dose3 Received        &  446.0 &     0.136 &    0.072 &    0.015 &     0.461 \\
Vaccination coverage  &  446.0 &    81.013 &   22.759 &   21.360 &   196.690 \\
BCG coverage          &  446.0 &    75.959 &   31.371 &    4.000 &   220.000 \\
Polio coverage   

<ipython-input-255-a0e93a3389ef>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(filtered_df[['Bolsonaro Share 22', 'Far-left 18', 'Far-right 22', 'Dose1 Applied', 'Dose2 Applied', 'Dose3 Applied',
